In [1]:
#Librerias utilizadas
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import datetime
from collections import Counter
%matplotlib inline
import matplotlib.pyplot as plt
nltk.download("punk")
nltk.download("stopwords")

[nltk_data] Error loading punk: Package 'punk' not found in index
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jogabell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Lectura de datos de la plataforma HIPEGIve

In [6]:
data_names = pd.read_csv('data/names_tokenized_and_cleaned.csv')

In [7]:
names_hipe = data_names[['0','1','2','3','4']]

In [10]:
data = names_hipe.values.tolist()

In [21]:
def get_clean_from_nan(name_list):
    clean_names=[]
    for e in name_list:
        clean_name=[]
        for i in e:            
            if not type(i)==float:
                clean_name.append(i)
        clean_names.append(clean_name)
    return clean_names

In [22]:
clean_names = get_clean_from_nan(data)

#### Lescuta y limpieza de base de datos de apellidos hispanos 

In [ ]:
def get_names_with_hispanic_surnames(lol):
    
    

In [2]:
names = pd.read_csv('data/hispanic_surnames.csv')

In [3]:
names

,hispanic_surnames
0,GARCIA
1,RODRIGUEZ
2,HERNANDEZ
3,MARTINEZ
4,LOPEZ
5,GONZALEZ
6,PEREZ
7,SANCHEZ
8,RAMIREZ
9,TORRES
